# Building AI Agents with LangGraph

## What is LangGraph?

**LangGraph** is a framework for building stateful, multi-step AI agent applications as **graphs**. It is part of the LangChain ecosystem but designed specifically for agent orchestration where you need fine-grained control over execution flow.

Unlike simple chain-based approaches, LangGraph lets you define **cyclic graphs** — meaning your agents can loop, retry, and route dynamically based on LLM decisions.

### Key Concepts at a Glance

| Concept | Description |
|---------|-------------|
| **State** | A shared data structure (typically a `TypedDict`) that flows through the graph. Every node reads from and writes to this state. |
| **Nodes** | Python functions that perform work — call an LLM, run a tool, transform data. Each node receives the current state and returns a partial update. |
| **Edges** | Connections between nodes. Can be *normal* (always follow) or *conditional* (route based on state). |
| **Tools** | Functions the LLM can call. Defined with the `@tool` decorator and bound to the model via `bind_tools()`. |
| **Compile & Run** | Once the graph is defined, you `.compile()` it into a runnable. Then `.invoke()` or `.stream()` to execute. |

### What you'll build in this notebook

1. Understand each core component with standalone examples
2. Build a **ReAct agent** (single agent + tools)
3. Add **memory & persistence** for multi-turn conversations
4. Build a **multi-agent research team** with a supervisor pattern

---
## Section 1: Setup

In [ ]:
# Install required packages
%pip install -q langgraph langchain langchain-openai langchain-community

In [ ]:
import os
import getpass

# Set your OpenAI API key.
# The notebook reads from the environment first; if not set, it prompts you.
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("API key is configured.")

---
## Section 2: Core Components Deep Dive

### 2.1 — State

The **state** is the central data structure that every node in your graph reads from and writes to. You define it as a `TypedDict`.

A critical concept is **reducers**. When a node returns a partial state update, the reducer determines *how* that update is merged into the existing state.

- **Default behavior**: the new value **overwrites** the old value.
- **`add_messages` reducer**: instead of overwriting, it **appends** new messages to the existing list. This is essential for chat-based agents where you want to accumulate a conversation history.

You apply a reducer using `typing.Annotated`:

```python
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]  # append, don't overwrite
    counter: int                              # plain overwrite
```

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

# Define a simple state with the add_messages reducer
class DemoState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int

# Simulate what the reducer does:
from langchain_core.messages import HumanMessage, AIMessage

existing = [HumanMessage(content="Hello")]
new = [AIMessage(content="Hi there!")]

# add_messages appends instead of replacing
merged = add_messages(existing, new)
for m in merged:
    print(f"{m.type}: {m.content}")

### 2.2 — Nodes

A **node** is simply a Python function that:
1. Receives the current `state` dict as its argument.
2. Does some work (call an LLM, run a tool, transform data).
3. Returns a **partial state update** — a dict containing only the keys you want to update.

```python
def my_node(state: AgentState) -> dict:
    # read from state
    messages = state["messages"]
    # do work ...
    return {"messages": [new_message]}  # partial update
```

The returned dict is merged into the state using the reducers defined in your `TypedDict`.

In [ ]:
# Example: a node that increments a counter and adds a message
def greeter_node(state: DemoState) -> dict:
    """A simple node that greets and increments a counter."""
    count = state.get("counter", 0) + 1
    return {
        "messages": [AIMessage(content=f"Greeting #{count}!")],
        "counter": count,
    }

# We'll wire this into a graph shortly. For now, let's test the logic:
result = greeter_node({"messages": [], "counter": 0})
print(result)

### 2.3 — Edges

Edges define how execution flows between nodes.

| Edge type | Method | Description |
|-----------|--------|-------------|
| Normal | `graph.add_edge("a", "b")` | Always go from node A to node B |
| Conditional | `graph.add_conditional_edges("a", routing_fn, mapping)` | Call `routing_fn(state)` to decide which node to go to next |
| Entry | `graph.add_edge(START, "a")` | Where the graph begins |
| Terminal | `graph.add_edge("a", END)` | Where the graph ends |

`START` and `END` are special sentinel nodes imported from `langgraph.graph`.

Conditional edges are the backbone of agent loops — they let the LLM decide whether to call a tool, route to another agent, or finish.

In [ ]:
from langgraph.graph import StateGraph, START, END

# Build a tiny graph: START -> greeter -> END
tiny_graph = StateGraph(DemoState)
tiny_graph.add_node("greeter", greeter_node)
tiny_graph.add_edge(START, "greeter")
tiny_graph.add_edge("greeter", END)

app = tiny_graph.compile()

# Run it
result = app.invoke({"messages": [HumanMessage(content="Hi!")], "counter": 0})
print(f"Counter: {result['counter']}")
for m in result["messages"]:
    print(f"  {m.type}: {m.content}")

### 2.4 — Tools

Tools let the LLM take **actions** — search the web, look up data, perform calculations, etc.

In LangChain/LangGraph, you:
1. Define tools using the `@tool` decorator from `langchain_core.tools`.
2. Bind them to a chat model with `model.bind_tools([tool1, tool2])`.
3. The LLM will include `tool_calls` in its response when it wants to use a tool.
4. A **tool node** executes those calls and returns the results as `ToolMessage`s.

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# Define a mock tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

# Create a model and bind the tool
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools([add_numbers])

# Ask the model something that should trigger the tool
response = llm_with_tools.invoke("What is 42 + 58?")

# The model doesn't compute the answer directly — it emits a tool_call
print("Tool calls:", response.tool_calls)
print("Content:", response.content)

---
## Section 3: Build a Simple ReAct Agent

The **ReAct** (Reason + Act) pattern is the most common agent architecture:

1. The LLM **reasons** about what to do.
2. It decides to **act** by calling a tool (or to finish).
3. The tool result is fed back, and the loop repeats.

```
START -> agent -> (tool_calls?) --yes--> tools -> agent (loop)
                                --no---> END
```

Let's build this from scratch.

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode


# --- 1. Define mock tools ---

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city."""
    # Mock data — no external API needed
    weather_data = {
        "new york": "72°F, partly cloudy, humidity 55%",
        "london": "58°F, overcast with light rain, humidity 80%",
        "tokyo": "68°F, clear skies, humidity 45%",
        "paris": "63°F, sunny with occasional clouds, humidity 50%",
        "sydney": "81°F, sunny, humidity 40%",
    }
    return weather_data.get(
        city.lower(),
        f"Weather data not available for {city}. Try: New York, London, Tokyo, Paris, Sydney."
    )


@tool
def search(query: str) -> str:
    """Search for information on a topic. Returns a brief summary."""
    # Mock search results
    results = {
        "langgraph": "LangGraph is a library by LangChain for building stateful, multi-actor AI applications with LLMs as graphs.",
        "python": "Python is a high-level, general-purpose programming language created by Guido van Rossum, released in 1991.",
        "react agent": "ReAct (Reason + Act) agents interleave reasoning and action steps, letting LLMs decide when to call tools.",
    }
    query_lower = query.lower()
    for key, value in results.items():
        if key in query_lower:
            return value
    return f"Search results for '{query}': This is a mock search. In production, connect to a real search API."


tools = [get_weather, search]
print(f"Defined {len(tools)} tools: {[t.name for t in tools]}")

In [ ]:
# --- 2. Create the model with tools bound ---

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model_with_tools = model.bind_tools(tools)


# --- 3. Define the state ---

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]


# --- 4. Define the agent node ---

def agent_node(state: AgentState) -> dict:
    """Call the LLM with the current messages. It will decide whether to use tools."""
    response = model_with_tools.invoke(state["messages"])
    return {"messages": [response]}


# --- 5. Define the routing function ---

def should_continue(state: AgentState) -> str:
    """Check if the last message has tool calls. If yes, route to tools; otherwise, end."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END


# --- 6. Build the graph ---

graph = StateGraph(AgentState)

# Add nodes
graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(tools))  # ToolNode auto-executes tool calls

# Add edges
graph.add_edge(START, "agent")                          # Entry point
graph.add_conditional_edges("agent", should_continue)   # Agent decides: tools or END
graph.add_edge("tools", "agent")                        # After tools, go back to agent

# Compile
react_agent = graph.compile()

print("ReAct agent compiled successfully!")

In [ ]:
# --- 7. Visualize the graph ---

print(react_agent.get_graph().draw_mermaid())

In [ ]:
# --- 8. Run the agent! ---

result = react_agent.invoke({
    "messages": [HumanMessage(content="What's the weather like in Tokyo and Paris?")]
})

# Print the full conversation
for msg in result["messages"]:
    role = msg.type
    if role == "human":
        print(f"\nUser: {msg.content}")
    elif role == "ai":
        if msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"\nAgent -> Tool Call: {tc['name']}({tc['args']})")
        if msg.content:
            print(f"\nAgent: {msg.content}")
    elif role == "tool":
        print(f"  Tool ({msg.name}): {msg.content}")

In [ ]:
# Try a search query too
result2 = react_agent.invoke({
    "messages": [HumanMessage(content="Search for information about LangGraph.")]
})

for msg in result2["messages"]:
    if msg.type == "ai" and msg.content:
        print(f"Agent: {msg.content}")

---
## Section 4: Adding Memory & Persistence

By default, each `.invoke()` call is stateless — the agent has no memory of previous conversations.

LangGraph solves this with **checkpointers**. A checkpointer saves the graph state after each step. When you invoke the graph with the same `thread_id`, it loads the previous state and continues from where it left off.

**Short-term memory** (within a thread): The agent remembers everything said in the current conversation thread.

**Long-term memory** (across threads): Requires external storage (database, vector store). Not built into the basic checkpointer, but LangGraph supports custom checkpointer backends for this.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Create a checkpointer (in-memory for this demo)
memory = MemorySaver()

# Recompile the same graph with the checkpointer attached
react_agent_with_memory = graph.compile(checkpointer=memory)

print("Agent recompiled with memory.")

In [ ]:
# Conversation turn 1 — use a thread_id to track this conversation
config = {"configurable": {"thread_id": "1"}}

result1 = react_agent_with_memory.invoke(
    {"messages": [HumanMessage(content="What is the weather in London?")]},
    config=config
)

# Show the agent's final answer
print("Turn 1 answer:")
print(result1["messages"][-1].content)

In [ ]:
# Conversation turn 2 — same thread_id, so it remembers turn 1
result2 = react_agent_with_memory.invoke(
    {"messages": [HumanMessage(content="How about New York? Is it warmer than the city I just asked about?")]},
    config=config  # Same thread_id!
)

print("Turn 2 answer:")
print(result2["messages"][-1].content)

In [ ]:
# Verify: the agent's state has the full conversation history
print(f"Total messages in thread: {len(result2['messages'])}")
print("\n--- Full conversation ---")
for msg in result2["messages"]:
    if msg.type == "human":
        print(f"\nUser: {msg.content}")
    elif msg.type == "ai" and msg.content:
        print(f"Agent: {msg.content}")

In [ ]:
# Different thread = fresh conversation, no memory of previous thread
config_new_thread = {"configurable": {"thread_id": "2"}}

result_new = react_agent_with_memory.invoke(
    {"messages": [HumanMessage(content="Which city did I just ask about?")]},
    config=config_new_thread
)

print("New thread answer (no memory of thread 1):")
print(result_new["messages"][-1].content)

---
## Section 5: Multi-Agent System — Research Team

Now let's build something more advanced: a **supervisor-based multi-agent system**.

### Architecture

```
START -> Supervisor --"researcher"--> Researcher -> Supervisor
                   --"writer"------> Writer     -> Supervisor
                   --"FINISH"------> END
```

- **Supervisor**: An LLM that reads the conversation and decides which worker to route to next (or to finish).
- **Researcher**: Has access to a search tool. Gathers information.
- **Writer**: Takes the research and produces a polished, well-structured output.

This pattern is powerful because each agent can be specialized, and the supervisor provides dynamic orchestration.

In [ ]:
from typing import Annotated, TypedDict, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode


# --- 1. Define the shared state ---

class TeamState(TypedDict):
    messages: Annotated[list, add_messages]
    next: str  # Which agent to route to next


# --- 2. Define tools for the Researcher ---

@tool
def research_search(query: str) -> str:
    """Search for research information on a topic."""
    # Expanded mock search with richer data
    knowledge_base = {
        "artificial intelligence": (
            "AI is a branch of computer science focused on creating systems that can perform tasks "
            "requiring human intelligence. Key areas include machine learning, NLP, computer vision, "
            "and robotics. The global AI market is projected to reach $1.8 trillion by 2030. "
            "Major players include OpenAI, Google DeepMind, Anthropic, and Meta AI."
        ),
        "climate change": (
            "Climate change refers to long-term shifts in global temperatures and weather patterns. "
            "Human activities, particularly burning fossil fuels, have been the main driver since the 1800s. "
            "The Paris Agreement aims to limit warming to 1.5°C. Renewable energy adoption is accelerating, "
            "with solar and wind now cheaper than coal in most regions."
        ),
        "quantum computing": (
            "Quantum computing leverages quantum mechanical phenomena like superposition and entanglement "
            "to process information. IBM, Google, and startups like IonQ are leading development. "
            "Potential applications include drug discovery, cryptography, and optimization problems."
        ),
    }
    query_lower = query.lower()
    for key, value in knowledge_base.items():
        if key in query_lower:
            return value
    return (
        f"Research on '{query}': This is a rapidly evolving field with significant recent developments. "
        f"Key trends include increased automation, global collaboration, and interdisciplinary approaches."
    )


@tool
def get_statistics(topic: str) -> str:
    """Get key statistics and data points about a topic."""
    stats = {
        "ai": "Global AI market: $196B (2023), projected $1.8T by 2030. AI adoption rate: 35% of companies.",
        "climate": "Global temp rise: +1.1°C since pre-industrial. CO2 levels: 421 ppm. Renewable energy: 30% of global electricity.",
        "quantum": "Quantum computing market: $1.3B (2024). Qubits achieved: 1,000+ (IBM). Estimated practical advantage: 2028-2032.",
    }
    topic_lower = topic.lower()
    for key, value in stats.items():
        if key in topic_lower:
            return value
    return f"Key statistics for '{topic}' are currently being compiled. Check specialized databases for the latest figures."


research_tools = [research_search, get_statistics]
print("Research tools defined.")

In [ ]:
# --- 3. Define the Supervisor node ---

supervisor_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

SUPERVISOR_SYSTEM_PROMPT = """You are a supervisor managing a research team. Your team members are:

- **researcher**: Gathers information and data using search tools. Send work here when you need facts, data, or research.
- **writer**: Writes polished, well-structured content based on available research. Send work here when enough research has been gathered.

Given the conversation so far, decide which team member should act next, or if the task is complete.

Respond with ONLY one of: "researcher", "writer", or "FINISH".

Guidelines:
- Start by sending the task to the researcher to gather information.
- Once sufficient research is available, send it to the writer.
- After the writer produces output, respond with FINISH.
- If the user's request is simple and doesn't need research, go directly to the writer."""


def supervisor_node(state: TeamState) -> dict:
    """The supervisor decides which agent to route to next."""
    messages = [
        SystemMessage(content=SUPERVISOR_SYSTEM_PROMPT),
        *state["messages"],
        HumanMessage(content="Who should act next? Respond with ONLY: researcher, writer, or FINISH.")
    ]
    response = supervisor_llm.invoke(messages)
    decision = response.content.strip().lower().replace('"', '').replace("'", "")

    # Normalize the decision
    if "researcher" in decision:
        next_agent = "researcher"
    elif "writer" in decision:
        next_agent = "writer"
    else:
        next_agent = "FINISH"

    return {
        "next": next_agent,
        "messages": [AIMessage(content=f"[Supervisor] Routing to: {next_agent}")]
    }


print("Supervisor node defined.")

In [ ]:
# --- 4. Define the Researcher node ---

researcher_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(research_tools)

RESEARCHER_SYSTEM_PROMPT = """You are a research specialist. Your job is to gather comprehensive information
about the topic using your available tools (research_search, get_statistics).

Always use your tools to find information. Summarize your findings clearly."""


def researcher_node(state: TeamState) -> dict:
    """The researcher gathers information using tools."""
    messages = [SystemMessage(content=RESEARCHER_SYSTEM_PROMPT), *state["messages"]]
    response = researcher_llm.invoke(messages)

    # If the LLM wants to call tools, execute them inline for simplicity
    if response.tool_calls:
        tool_map = {t.name: t for t in research_tools}
        tool_results = []
        for tc in response.tool_calls:
            result = tool_map[tc["name"]].invoke(tc["args"])
            tool_results.append(f"{tc['name']}: {result}")

        summary = "\n".join(tool_results)
        return {
            "messages": [AIMessage(content=f"[Researcher] Findings:\n{summary}")]
        }
    else:
        return {
            "messages": [AIMessage(content=f"[Researcher] {response.content}")]
        }


print("Researcher node defined.")

In [ ]:
# --- 5. Define the Writer node ---

writer_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

WRITER_SYSTEM_PROMPT = """You are a professional writer. Based on the research findings in the conversation,
produce a polished, well-structured summary. Use clear headings, bullet points, and concise language.
Write for a general audience."""


def writer_node(state: TeamState) -> dict:
    """The writer produces polished output from research."""
    messages = [SystemMessage(content=WRITER_SYSTEM_PROMPT), *state["messages"]]
    response = writer_llm.invoke(messages)
    return {
        "messages": [AIMessage(content=f"[Writer] {response.content}")]
    }


print("Writer node defined.")

In [ ]:
# --- 6. Build the multi-agent graph ---

def route_from_supervisor(state: TeamState) -> str:
    """Route based on the supervisor's decision."""
    next_agent = state.get("next", "FINISH")
    if next_agent == "FINISH":
        return END
    return next_agent


# Create the graph
team_graph = StateGraph(TeamState)

# Add nodes
team_graph.add_node("supervisor", supervisor_node)
team_graph.add_node("researcher", researcher_node)
team_graph.add_node("writer", writer_node)

# Entry: always start at supervisor
team_graph.add_edge(START, "supervisor")

# Supervisor routes conditionally
team_graph.add_conditional_edges(
    "supervisor",
    route_from_supervisor,
    {"researcher": "researcher", "writer": "writer", END: END}
)

# Workers always report back to supervisor
team_graph.add_edge("researcher", "supervisor")
team_graph.add_edge("writer", "supervisor")

# Compile
research_team = team_graph.compile()

print("Multi-agent research team compiled!")

In [ ]:
# Visualize the multi-agent graph
print(research_team.get_graph().draw_mermaid())

In [ ]:
# --- 7. Run the research team ---

result = research_team.invoke({
    "messages": [
        HumanMessage(content="Write a brief report on the current state of artificial intelligence.")
    ],
    "next": ""
})

# Print the full conversation flow
print("=" * 70)
print("RESEARCH TEAM — FULL CONVERSATION FLOW")
print("=" * 70)

for msg in result["messages"]:
    if msg.type == "human":
        print(f"\n{'─'*50}")
        print(f"USER: {msg.content}")
    elif msg.type == "ai" and msg.content:
        # Determine which agent spoke
        print(f"\n{'─'*50}")
        print(msg.content[:500])  # Truncate very long outputs for readability
        if len(msg.content) > 500:
            print("... [truncated for display]")

In [ ]:
# Show the writer's final output in full
print("=" * 70)
print("FINAL OUTPUT")
print("=" * 70)

for msg in reversed(result["messages"]):
    if msg.type == "ai" and msg.content.startswith("[Writer]"):
        print(msg.content.replace("[Writer] ", ""))
        break

---
## Section 6: Key Takeaways

### When to use LangGraph

LangGraph is the right choice when you need:

- **Fine-grained control** over agent execution flow (custom routing, branching, loops)
- **Complex state management** beyond simple message passing
- **Cyclic graphs** — agents that can loop, retry, and self-correct
- **Multi-agent orchestration** with custom coordination patterns
- **Built-in persistence and memory** via checkpointers
- **Streaming** and step-by-step observability into agent behavior

### LangGraph vs. higher-level frameworks

| | LangGraph | CrewAI / AutoGen |
|---|-----------|------------------|
| **Abstraction level** | Low-level, graph-based | High-level, role-based |
| **Control** | Full control over every edge and node | More opinionated, less customizable |
| **Learning curve** | Steeper — you build the graph yourself | Gentler — define roles and goals |
| **Flexibility** | Arbitrary graph topologies | Mostly linear/sequential flows |
| **State management** | Custom TypedDict with reducers | Built-in but less flexible |
| **Best for** | Production systems, complex workflows | Prototyping, simpler multi-agent setups |

### What we covered

1. **Core concepts**: State, Nodes, Edges, Tools — the building blocks of every LangGraph application.
2. **ReAct agent**: The fundamental single-agent + tools loop.
3. **Memory**: Using `MemorySaver` and thread IDs for multi-turn conversations.
4. **Multi-agent systems**: The supervisor pattern for coordinating specialized agents.

### Next steps

- Explore **human-in-the-loop** with `interrupt_before` / `interrupt_after` to pause the graph for user approval.
- Try **streaming** with `.stream()` and `.astream_events()` for real-time output.
- Use **LangGraph Platform** (LangGraph Cloud) to deploy agents as APIs.
- Implement **persistent checkpointers** (e.g., SQLite, PostgreSQL) for production memory.